# 1 Introdução ao Altair

[Altair](https://altair-viz.github.io/) é uma biblioteca declarativa de visualização estatística para Python. O Altair oferece uma gramática concisa e poderosa para construir rapidamente uma multiplicidade de gráficos estatísticos.

Com o termo *declarativo*, queremos dizer que você pode usar um alto nível de especificação *do quê* você quer que a visualização tenha, em termos de *dados*, *marcas gráficas*, e *canais de codificação*, ao invés de ter que especificr *como* implementar a visualização em termos de "loops de for",comandos de desenho de baixo nível, e *etc*. A principal ideia é que você declare ligações entre campos de dados e canais de codificação visual, como o eixo-x, eixo-y, *etc*. O resto dos detelhes de plot são resolvidos automaticamente. Construir nessa ideia de plotagem declarativa pode criar uma multiplicidade de visualizações, das simples à mais sofisticadas, usando uma gramática concisa.  

Altair é baseado no [Vega-Lite](https://vega.github.io/vega-lite/), uma gramática de alto nível de gráficos interativos. Altair oferece uma [API (Application Programming Interface)](https://en.wikipedia.org/wiki/Application_programming_interface) bem projetada para Python que gera especificações para Vega-Lite no formato [JSON (JavaScript Object Notation)](https://en.wikipedia.org/wiki/JSON). Ambientes como o Jupyter NoteBooks, JupyterLab, e Colab podem então processar essa especificação e exibir diretamente no browser. Para aprender mais sobre a motivação e conceitos básicos por trás do Altair e Vega-Lite, assista [Vídeo de apresentação do Vega-Lite da OpenVisConf 2017](https://www.youtube.com/watch?v=9uaHRWj04D4).

Esse notebook vai guiá-lo pelo processo básico de criar visualizações em Altair. Em primeiro lugar, você vai precisar conferir que tem o pacote do Altair e suas dependências instalados (para mais, veja [Documentação de instalação do Altair](https://altair-viz.github.io/getting_started/installation.html)), ou que você esteja usando um ambiente notebook que inclua as dependências pré-instaladas.

_Esse notebook é parte do [data visualization curriculum](https://github.com/uwdata/visualization-curriculum)._


## 1.1 Importações

Para começar, nós precisamos importar as bibliotecas necessárias: Pandas para os data frames e Altair para visualização.

In [67]:
import pandas as pd
import altair as alt

## 1.2 Renderizadores

Dependendo do seu ambiente, você pode precisar de um [rendererizador](https://altair-viz.github.io/user_guide/display_frontends.html) para Altair. Se você está usando __JupyterLab__, __Jupyter Notebook__, ou __Google Colab__ com conexão à Internet você não deveria precisar fazer nada. Caso contrário, por favor leia a documentação [Exibindo Gráficos do Altair](https://altair-viz.github.io/user_guide/display_frontends.html).

## 1.3 Dados

Os dados no Altair são estruturados em torno do data frame do Pandas, que consiste em um conjunto de *colunas* de dados nomeadas. Também nos referiremos regularmente às colunas de dados como *campos* de dados.

Ao usar o Altair, os conjuntos de dados geralmente são fornecidos como data frames. Alternativamente, o Altair também pode aceitar uma URL para carregar um conjunto de dados acessível pela rede. Como veremos, as colunas nomeadas do data frame são uma peça essencial para a criação de gráficos com o Altair.

Frequentemente, usaremos conjuntos de dados do repositório [vega-datasets](https://github.com/vega/vega-datasets). Alguns desses conjuntos de dados estão disponíveis diretamente como DataFrames do Pandas:

In [68]:
from vega_datasets import data  # importa o módulo vega_datasets
cars = data.cars()              # carrega os dados "cars" como um data frame do Pandas
cars.head()                     # exibe as primeiras cinco linhas

,Name,Miles_per_Gallon,Cylinders,Displacement,Horsepower,Weight_in_lbs,Acceleration,Year,Origin
0,chevrolet chevelle malibu,18.0,8,307.0,130.0,3504,12.0,1970-01-01,USA
1,buick skylark 320,15.0,8,350.0,165.0,3693,11.5,1970-01-01,USA
2,plymouth satellite,18.0,8,318.0,150.0,3436,11.0,1970-01-01,USA
3,amc rebel sst,16.0,8,304.0,150.0,3433,12.0,1970-01-01,USA
4,ford torino,17.0,8,302.0,140.0,3449,10.5,1970-01-01,USA


Os conjuntos de dados na coleção vega-datasets também podem ser acessados por meio de URLs:

In [69]:
data.cars.url

'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/cars.json'

URLs de conjuntos de dados podem ser passadas diretamente ao Altair (considerando formatos compatíveis, como JSON e [CSV](https://en.wikipedia.org/wiki/Comma-separated_values)) ou carregadas em um data frame do Pandas da seguinte maneira:

In [70]:
pd.read_json(data.cars.url).head() # carrega dados em formato JSON em um DataFrame

,Name,Miles_per_Gallon,Cylinders,Displacement,Horsepower,Weight_in_lbs,Acceleration,Year,Origin
0,chevrolet chevelle malibu,18.0,8,307.0,130.0,3504,12.0,1970-01-01,USA
1,buick skylark 320,15.0,8,350.0,165.0,3693,11.5,1970-01-01,USA
2,plymouth satellite,18.0,8,318.0,150.0,3436,11.0,1970-01-01,USA
3,amc rebel sst,16.0,8,304.0,150.0,3433,12.0,1970-01-01,USA
4,ford torino,17.0,8,302.0,140.0,3449,10.5,1970-01-01,USA


Para mais informações sobre data frames - e algumas transformações úteis para preparar data frames do Pandas para criação de gráficos com o Altair! - veja [documentação sobre Especificação de Dados com o Altair](https://altair-viz.github.io/user_guide/data.html).

### 1.3.1 Dados Climáticos

A visualização estatística no Altair começa com ["tidy"](http://vita.had.co.nz/papers/tidy-data.html) data frames. Aqui, começaremos criando um data frame simples (`df`) contendo a precipitação média (`precip`) para uma determinada cidade (`city`) e mês (`month`):

In [71]:
df = pd.DataFrame({
    'city': ['Seattle', 'Seattle', 'Seattle', 'New York', 'New York', 'New York', 'Chicago', 'Chicago', 'Chicago'],
    'month': ['Apr', 'Aug', 'Dec', 'Apr', 'Aug', 'Dec', 'Apr', 'Aug', 'Dec'],
    'precip': [2.68, 0.87, 5.31, 3.94, 4.13, 3.58, 3.62, 3.98, 2.56]
})

df

,city,month,precip
0,Seattle,Apr,2.68
1,Seattle,Aug,0.87
2,Seattle,Dec,5.31
3,New York,Apr,3.94
4,New York,Aug,4.13
5,New York,Dec,3.58
6,Chicago,Apr,3.62
7,Chicago,Aug,3.98
8,Chicago,Dec,2.56


## 1.4 O Objeto Gráfico

O objeto fundamental no Altair é o Gráfico (`chart`), que recebe um data frame como único argumento:

In [72]:
chart = alt.Chart(df)

Até agora, definimos o objeto gráfico (`chart`) e passamos a ele o data frame simples que geramos acima. Ainda não dissemos ao `chart` o que fazer com os dados.

## 1.5 Marcas e Codificações
Com um objeto gráfico em mãos, agora podemos especificar como gostaríamos que os dados fossem visualizados. Primeiro, indicamos o tipo de marca gráfica (forma geométrica) que queremos usar para representar os dados. Podemos definir o atributo de `mark` do objeto gráfico usando os métodos `Chart.mark_*`.

Por exemplo, podemos mostrar o dado como um ponto usando `Chart.mark_point()`:

In [73]:
alt.Chart(df).mark_point()

alt.Chart(...)

Aqui, a renderização consiste em um ponto por linha do conjunto de dados, todos sobrepostos uns aos outros, já que ainda não especificamos as posições desses pontos.

Para separar visualmente os pontos, podemos mapear diversos canais de codificação (ou apenas canais) para os campos do conjunto de dados. Por exemplo, poderíamos codificar o campo `city` dos dados usando o canal y, que representa a posição dos pontos no eixo y. Para especificar isso, use o método `encode`:

In [74]:
alt.Chart(df).mark_point().encode(
  y='city',
)

alt.Chart(...)

O método `encode` constrói um mapeando valor-chave entre os canais de codificação(como o `x`,`y`, `color`, `shape`, `size`, etc.) no campos do conjunto de dados, acessado pelo campo `name`. Para as tabelas de dados do Pandas, o Altair indica automaticamente um tipo de apropriado dado para a coluna mapeada, que, nesse caso, é o tipo *nominal*, indicando valores categóricos e não ordenados.

Embora tenhamos agora separado os dados por um atributo, nós ainda temos multiplos pontos sobrepostos dentro de cada categoria. Vamos separar ainda mais esse pontos adicionando uma codificação no canal `x`. Mapeado para o campo `'precip'`:

In [75]:
alt.Chart(df).mark_point().encode(
    x='precip',
    y='city'
)

alt.Chart(...)

_Seattle exibe tanto os meses mais chuvosos quanto os menos chuvosos!_

O tipo de dado do campo `'precip'` novamente atribuído autimaticamente através de uma inferência do Altair, sendo tratado como um campo *quantitativo*(que é um número real). Observamos que as linhas de grade e os títulos apropriados dos eixos são adicionados automaticamente.

Acima, especificamos pares chave-valor usando argumentos nomeados `(x='precip')`. Além disso, o Altair oferece métodos de construção para definições de codificação, utilizando a sintaxe `alt.X('precip')`. Essa alternativa é útil para fornecer mais parâmetros a uma codificação, como veremos mais adiante neste notebook.


In [76]:
alt.Chart(df).mark_point().encode(
    alt.X('precip'),
    alt.Y('city')
)

alt.Chart(...)


Os dois estilos de especificar codificações podem ser combinados: `x='precip'`, `alt.Y('city')` também é uma entrada válida para a função encode.

Nos exemplos acima, o tipo de dado para cada campo é inferido automaticamente com base em seu tipo dentro do data frame do Pandas. Também podemos indicar explicitamente o tipo de dado ao Altair, anotando o nome do campo:



*   `'b:N'` indica um tipo nominal (dados categóricos não ordenados);
*   `'b:O'` indica um tipo ordinal (dados ordenados por classificação);
*   `'b:Q'` indica um tipo quantitativo (dados numéricos com magnitudes significativas); e
*   `'b:T'` indica um tipo temporal (dados de data/tempo).

Por exemplo, `alt.X('precip:N')`.

A anotação explícita dos tipos de dados é necessária quando os dados são carregados diretamente de uma URL externa pelo Vega-Lite (ignorando completamente o Pandas) ou quando desejamos usar um tipo que difere do tipo inferido automaticamente.

O que você acha que acontecerá com o nosso gráfico acima se tratarmos `'precip'` como uma variável nominal ou ordinal, em vez de uma variável quantitativa? _Modifique o código acima e descubra!_

Analisaremos mais detalhadamente os tipos de dados e os canais de codificação no próximo notebook do [currículo de visualização de dados](https://github.com/uwdata/visualization-curriculum#data-visualization-curriculum).


## 1.6 Tranformação de Dados: Agregação

Para permitir maior flexibilidade na visualização dos dados,  Altair tem uma sintaxe própria para agregação de dados. Por exemplo, podemos computar no exemplo da base de dados "seattle_temps" a temperatura média a cada mês ao invés de calcular o gráfico com todos os dados que implica em uma compreensão mais complicada:

Nesse caso preciamos fazer um tratamento nos dados já que a base de dados tem mais de 500 linhas então usaremos a função seguinte para que não tenhamos nenhum limite de linhas na análise

In [77]:
alt.data_transformers.disable_max_rows()


DataTransformerRegistry.enable('default')

Aqui exibimos o gráfico de todas as temperaturas durante um ano, espaçadas por uma hora, na cidade de Seattle no ano de 2010. No entanto é possível perceber que não é tão simples entender o gráfico devido à grande quantidade de pontos exibidos.

In [78]:
alt.data_transformers.disable_max_rows()
temper = data.seattle_temps()
chart = alt.Chart(temper).mark_line().encode(
    x = alt.X('date'),
    y = alt.Y('temp')
)
chart

alt.Chart(...)

Assim, faremos um tratamento nos dados utilizando funções próprias do Altair que possibilitam separar as temperaturas emapenas uma temperatura média a cada mÊs possibilitando uma melhor compreensão do gráfico.

In [79]:
alt.Chart(temper).mark_line().encode(
    x = 'month(date):T',
    y = 'mean(temp):Q')

alt.Chart(...)

Altair tem uma grande variedade de funções de agregação, incluindo `count`, `min` (mínimo), `max` (máximo), `average`, `median`, and `stdev` (desvio padrão). Em um notebook futuro, nós vamos ter um passeio por transformação de dados, incluindo agregação, ordenação, filtragem, e criação de novos campos derivados utilizando fórmulas de cálculo.

## 1.7 Mudando o Tipo de Marcação
Vamos dizer que agora queremos representar a média de temperatura de cada mês por barras ao invés de pontos conectados por uma linha. Nós podemos fazer isso substituindo `Chart.mark_point` por `Chart.mark_bar`:


In [80]:
alt.Chart(temper).mark_bar().encode(
    x='average(temp)',
    y='month(date)'
)

alt.Chart(...)

Como os meses estão no eixo y isso resulta em um gráfico de barras horizontais, caso seja desejado um gráfico de barras verticais basta inverter o eixo x com o y

In [81]:
alt.Chart(temper).mark_bar().encode(
    x='month(date)',
    y='average(temp)'
)

alt.Chart(...)

## 1.8 Customizando a Visualização

Por padrão, o Altair/Vega-Lite faz algumas escolhas sobre as propriedades de visualizações. Porém, essas propriedades podem ser alteradas utilizando métodos para customizar a aparência da visualização. Por exemplo, é possível especificar os títulos dos eixos utilizando  o atributo `axis`. Também é possivel modificar as propriedades de escala utilizando o atributo `scale` e especificar a cor dos ícones do gráfico através do atributo `color` do   `Chart.mark_*`. Porém, apenas algumas cores válidas por string estão disponíveis, confira em: [CSS color string](https://developer.mozilla.org/en-US/docs/Web/CSS/color_value):

No exemplo a seguir, estaremos customizando: Titulo de ambos os eixos, a escala do eixo Y, e o intervalo de ambos os eixos, cor do ícone.

In [82]:
alt.Chart(df).mark_point(color='firebrick').encode(
  alt.X('precip', scale=alt.Scale(type='log'), axis=alt.Axis(title='Precipitação [escala logaritimica]')),
  alt.Y('city', axis=alt.Axis(title='Cidade')),
)

alt.Chart(...)

O  próximo módulo exibirá diversas opções disponiveis para customizar graficos. Como: eixos, escalas e legendas.

## 1.9 Múltiplas Visualizações

Como vimos acima, o objeto Altair `Chart` representa um gráfico de linha, barra, *etc.* Mas e se forem necessários diagramas mais complicados, envolvendo múltiplos gráficos ou camadas? Usando um conjunto de operadores de **composição de visualizações**, o Altair consegue fazer múltiplas definições de gráficos e combiná-las para criar *visualizações* mais complexas.

Como ponto de partida, vamos observar um exemplo abaixo:

In [83]:
alt.Chart(cars).mark_line().encode(
    alt.X('Year'),
    alt.Y('average(Miles_per_Gallon)')
)

alt.Chart(...)

Podemos ter uma melhor visualização desse gráfico adicionando uma marca `circular` em cada ponto que representa a eficiência média dos motores em cada ano. Para isso, vamos definir cada gráfico separadamente. Primeiramente, plotamos um gráfico de linha, em seguida um gráfico de dispersão. Daí, usamos o operador de **layer** para combinar os dois. Aqui utilizamos o operador abreviado `+` para a junção de camadas.

In [84]:
line = alt.Chart(cars).mark_line().encode(
    alt.X('Year'),
    alt.Y('average(Miles_per_Gallon)')
)

point = alt.Chart(cars).mark_circle().encode(
    alt.X('Year'),
    alt.Y('average(Miles_per_Gallon)')
)

line + point

alt.LayerChart(...)

Também podemos criar esse tipo de diagrama *reutilizando* e *modificando* um gráfico anteriormente definido. Podemos primeiramente criar um gráfico de linha, e em seguida usar o método `mark_circle` para gerar um tipo de gráfico diferente.

In [85]:
mpg = alt.Chart(cars).mark_line().encode(
    alt.X('Year'),
    alt.Y('average(Miles_per_Gallon)')
)

mpg + mpg.mark_circle()

alt.LayerChart(...)

Porém, e se necessitarmos colocar, junto a essa representação, um outro gráfico, como um de potência média dos motores ao longo do tempo? Para isso, basta usarmos um operador de *concatenação*. Dessa forma, podemos colocar múltiplos gráficos lado a lado. Aqui utilizaremos a barra `|` para realizar a concatenação na horizontal.

In [86]:
hp = alt.Chart(cars).mark_line().encode(
    alt.X('Year'),
    alt.Y('average(Horsepower)')
)

(mpg + mpg.mark_circle()) | (hp + hp.mark_circle())

alt.HConcatChart(...)

_Conseguimos observar um crescimento da eficiência média e um decrescimento na potência média._

O tópico seguinte será focado em composição de visualizações, incluindo não somente sobreposição de camadas e concatenação, mas também os operadores `facet`, para dividir os dados, e `repeat`, para gerar de forma concisa gráficos concatenados a partir de um modelo.

## 1.10 Interatividade

Além de plotagem básica de gráficos e composição visual, uma das funções mais legais do Altair e do Vega-Lite é suportar interatividade.

Para criar um *plot* interativo simples que tolera arrastar e dar *zoom*, podemos invocar o método `interactive()` do objeto `Chart`. No diagrama/*chart* abaixo, clique e arraste para se mover no plano, e role para dar *zoom*.

In [87]:
alt.Chart(cars).mark_point().encode(
    x='Horsepower',
    y='Miles_per_Gallon',
    color='Origin',
).interactive()

alt.Chart(...)

Ao passar o mouse sobre um ponto, vemos algumas informações em um quadro. Esse quadro se chama *tooltip*, e pode ser modificado. Para alterar o que é mostrado quando o mouse está em cima de um ponto, podemos usar o argumento `tooltip` ao invocar a função `encode:`

In [88]:
alt.Chart(cars).mark_point().encode(
    x='Horsepower',
    y='Miles_per_Gallon',
    color='Origin',
    tooltip=['Name', 'Origin'] # Mostra o nome (Name) e a origem (Origin) dos carros na tooltip
).interactive()

alt.Chart(...)

Para interações mais complexas, como por exemplo charts interligadas e filtragem atravessada, o Altair nos dá uma abstração de seleção para que possamos definir uma seleção interativa e depois associá-la a um elemento do chart. Vamos falar sobre isso em mais detalhes à frente.

Abaixo temos um exemplo de interatividade mais complexo. O histograma em cima mostra a quandidade de carros por ano e usa uma seleção interativa para modificar a opacidade dos pontos no *scatter plot* em baixo, que mostra potência em cavalos (Horsepower) contra kilometragem por litro (Mileage).

_Arraste um intervalo na chart superior e veja como afeta os pontos no scatter plot inferior. Conforme você for examinando o código, por favor, **não se preocupe se não faz sentido ainda!** Esse é um exemplo mais voltado a te inspirar, e vamos te preencher nos detalhes conforme você avança._

In [89]:
# cria uma seleção de intervalos no encodingo do eixo x
brush = alt.selection_interval(encodings=['x'])

# determina opacidade baseado no brush (intervalo selecionado)
opacity = alt.condition(brush, alt.value(0.9), alt.value(0.1))

# um histograma de carros por ano
# adiciona o brush para selecionar os carros com o tempo
overview = alt.Chart(cars).mark_bar().encode(
    alt.X('Year:O', timeUnit='year', # extrai a unidade de ano (year), trata como um número ordinal
      axis=alt.Axis(title=None, labelAngle=0) # sem título, e o nome do eixo não está angulado
    ),
    alt.Y('count()', title=None), #eixo y: faz contagem, sem título
    opacity=opacity
).add_params(
    brush      # adiciona o brush no chart
).properties(
    width=400, # define a largura do chart como 400 pixels
    height=50  # define a altura do chart como 50 pixels
)

# um scatter plot detalhado de potência em cavalos (Horsepower) vs. kilometragem por litro (mileage)
# muda a opacidade do ponto baseado no intervalo selecionado
detail = alt.Chart(cars).mark_point().encode(
    alt.X('Horsepower'),
    alt.Y('Miles_per_Gallon'),
    # define a opacidade baseado no intervalo selecionado
    opacity=opacity
).properties(width=400) # define a largura do chart para bater com a do primeiro

# concatena verticalmente (vconcat) os charts usando o operador &
overview & detail

alt.VConcatChart(...)

## 1.11 A parte: Examinando a Saída JSON

Como um API Python para Vega-Lite, o principal proprosito do Altair é converter especificações de plotagem para uma string JSON que esteja em conforme com o esquema do Vega-Lite. Usando o método Chart.to_json, conseguimos inspecionar a especificação JSON que Altair está exportando e enviando para o Vega-Lite:

In [90]:
chart = alt.Chart(df).mark_bar().encode(
    x='average(precip)',
    y='city',
)
print(chart.to_json())

{
  "$schema": "https://vega.github.io/schema/vega-lite/v5.20.1.json",
  "config": {
    "view": {
      "continuousHeight": 300,
      "continuousWidth": 300
    }
  },
  "data": {
    "name": "data-8e72c2f67818e64f2c6d729f1a903405"
  },
  "datasets": {
    "data-8e72c2f67818e64f2c6d729f1a903405": [
      {
        "city": "Seattle",
        "month": "Apr",
        "precip": 2.68
      },
      {
        "city": "Seattle",
        "month": "Aug",
        "precip": 0.87
      },
      {
        "city": "Seattle",
        "month": "Dec",
        "precip": 5.31
      },
      {
        "city": "New York",
        "month": "Apr",
        "precip": 3.94
      },
      {
        "city": "New York",
        "month": "Aug",
        "precip": 4.13
      },
      {
        "city": "New York",
        "month": "Dec",
        "precip": 3.58
      },
      {
        "city": "Chicago",
        "month": "Apr",
        "precip": 3.62
      },
      {
        "city": "Chicago",
        "month": "Aug",

Observe aqui que `encode(x='average(precip)')` foi expandido para uma estrutura JSON com o nome `field`, um `type` para os dados, e inclui um `aggregate` field. A declaração `encode(y='city')` foi expandido de forma semelhante.

Como visto anteriormente, a sintaxe abreviada do Altair inclui uma forma de especificar o tipo de campo:

In [91]:
x = alt.X('average(precip):Q')
print(x.to_json())

{
  "aggregate": "average",
  "field": "precip",
  "type": "quantitative"
}


Essa abreviação é equivalente a soletrar os atributos pelo nome:

In [92]:
x = alt.X(aggregate='average', field='precip', type='quantitative')
print(x.to_json())

{
  "aggregate": "average",
  "field": "precip",
  "type": "quantitative"
}


## 1.12 Publicando Uma Visualisação

Depois de visualizar seus dados, talvez você queira publicá-los em algum lugar na web. Isso pode ser feito diretamente usando o [pacote JavaScript vega-embed](https://github.com/vega/vega-embed). Um exemplo simples de um documento HTML autônomo pode ser gerado para qualquer gráfico usando o método `Chart.save`:

In [93]:
chart = alt.Chart(df).mark_bar().encode(
    x='average(precip)',
    y='city',
)
chart.save('chart.html')

O modelo básico em HTML produz uma saída semelhante a esta, onde a especificação JSON do seu gráfico, gerada pelo `Chart.to_json`, deve ser armazenada na variável `spec` do JavaScript:

In [94]:
%%html
<!DOCTYPE html>
<html>
  <head>
    <script src="https://cdn.jsdelivr.net/npm/vega@5"></script>
    <script src="https://cdn.jsdelivr.net/npm/vega-lite@4"></script>
    <script src="https://cdn.jsdelivr.net/npm/vega-embed@6"></script>
  </head>
  <body>
    <div id="vis"></div>
    <script>
      (function(vegaEmbed) {
        var spec = {
          "data": {"values": [{"a": "A", "b": 28}, {"a": "B", "b": 55}]},
          "mark": "bar",
          "encoding": {
            "x": {"field": "a", "type": "nominal"},
            "y": {"field": "b", "type": "quantitative"}
          }
        }; /* Exemplo de especificação JSON para um gráfico */

        var embedOpt = {"mode": "vega-lite"}; /* Opções para o embedding */

        function showError(el, error){
          el.innerHTML = ('<div style="color:red;">'
                          + '<p>JavaScript Error: ' + error.message + '</p>'
                          + "<p>This usually means there's a typo in your chart specification. "
                          + "See the javascript console for the full traceback.</p>"
                          + '</div>');
          throw error;
        }
        const el = document.getElementById('vis');
        vegaEmbed("#vis", spec, embedOpt)
          .catch(error => showError(el, error));
      })(vegaEmbed);
    </script>
  </body>
</html>


O método `Chart.save` fornece uma maneira conveniente de salvar tal saída HTML em um arquivo. Para mais informações sobre como incorporar Altair/Vega-Lite, veja a [documentação do projeto vega-embed](https://github.com/vega/vega-embed) .

## 1.13 Próximos Passos

🎉 Oba, você concluiu a introdução ao Altair! No próximo notebook, vamos nos aprofundar na criação de visualizações usando o modelo de tipos de dados, marcas gráficas e canais de codificação visual do Altair.